<h1 style='font-size:40px'> Ensemble Learning and Random Forests</h1>
<div>
    <ul style='font-size:20px'> 
        <li> 
            O aprendizado Ensemble consiste em fazer previsões embasadas em um conjunto de modelos, ao invés de um único. Essa técnica é inspirada no conceito de <em> sabedoria da multidão</em>.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Voting Classifiers</h2>
<div>
    <ul style='font-size:20px'> 
        <li> 
            Um classificador por voto é um conjunto de classificadores diferentes que operam como um só. A previsão final é a mais recorrente entre cada um dos algoritmos individuais.
        </li>
    </ul>
</div>

<p style='color:red'> Montar o código do VotingClassifier